# selecting the best model 

pandas 
numpy 
test_train_split
models: 1.logistic regression 
        2.svm
        3.Xgboost
        4. random forest 
        5. dedcision tree
grid search
random grid search 
evaluation matrics

In [8]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


In [9]:
from imblearn.over_sampling import SMOTE


In [10]:
data = pd.read_csv("/Users/sarthaktyagi/Desktop/projects /fraudDetection/data/cleaned_data")
data.shape
data.head(3)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,False,False,True,False,0.0,9839.64
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,False,False,True,False,0.0,1864.28
2,1,181.00,181.0,0.00,0.0,0.0,1,False,False,False,True,0.0,181.00


In [11]:
X= data.drop(columns=['isFraud'])
y = data.isFraud
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size= 0.2,random_state=42)

# applying smote 
sm = SMOTE(random_state=42)
X_train_res,y_train_res = sm.fit_resample(X_train,y_train)

In [12]:
X_train_res.head(4)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,15,9914.74,44248.0,34333.26,0.00,0.00,False,False,True,False,0.00,9914.74
1,20,6854.53,0.0,0.00,0.00,0.00,False,False,True,False,6854.53,6854.53
2,231,361211.80,0.0,0.00,489745.16,850956.95,True,False,False,False,361211.80,0.01
3,236,7083.51,0.0,0.00,0.00,0.00,False,False,True,False,7083.51,7083.51


In [13]:
X_train_res.head(6)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,15,9914.74,44248.00,34333.26,0.00,0.00,False,False,True,False,0.00,9914.74
1,20,6854.53,0.00,0.00,0.00,0.00,False,False,True,False,6854.53,6854.53
2,231,361211.80,0.00,0.00,489745.16,850956.95,True,False,False,False,361211.80,0.01
3,236,7083.51,0.00,0.00,0.00,0.00,False,False,True,False,7083.51,7083.51
4,143,218019.51,13045685.58,13263705.09,2438123.98,2220104.47,False,False,False,False,436039.02,436039.02
5,283,186134.83,26150.00,212284.83,425417.31,239282.48,False,False,False,False,372269.66,372269.66


In [14]:
from collections import Counter
print("before smote",Counter(y_train))
print("after smote",Counter(y_train_res))


before smote Counter({0: 5083526, 1: 6570})
after smote Counter({0: 5083526, 1: 5083526})


In [5]:
X.head(1)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,1,9839.64,170136.0,160296.36,0.0,0.0,False,False,True,False,0.0,9839.64


In [6]:
models = {
    "LogisticRegression": {
        "model": LogisticRegression(),
        "param": {
            "C": [0.01, 0.1, 1],
            "solver": ['liblinear'],
            "class_weight": [None, 'balanced'],
            "max_iter": [100, 200, 500, 1000]
        }
    },
    "DecisionTreeClassifier": {
        "model": DecisionTreeClassifier(),
        "param": {
            "criterion": ["gini", "entropy"],
            "max_depth": [5, 10],
            "min_samples_split": [1000, 2000]
        }
    },
    "RandomForestClassifier": {
        "model": RandomForestClassifier(),
        "param": {
            "n_estimators": [50, 100],
            "criterion": ["gini", "entropy"],
            "max_depth": [10, 15],
            "min_samples_split": [1000, 2000]
        }
    },
    "SupportVectorMachine": {
        "model": SVC(),
        "param": {
            "C": [0.1, 1],
            "kernel": ["linear", "rbf"],
            "gamma": ["scale", "auto"],
            "class_weight": [None, "balanced"]
        }
    },
    "GradientBoostingClassifier": {
        "model": GradientBoostingClassifier(),
        "param": {
            "loss": ["log_loss", "exponential"],
            "learning_rate": [0.1, 0.01],
            "n_estimators": [100, 150],
            "min_samples_split": [1000, 2000]
        }
    }
}

In [7]:
from sklearn.model_selection import GridSearchCV

best_results = {}

# Loop through your models dictionary
for name, m in models.items():
    print(f"Training {name}...")
    grid = GridSearchCV(m['model'], m['param'], cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)
    
    # Store best score and params
    best_results[name] = {
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    }

# To view results
from pprint import pprint
pprint(best_results)

Training LogisticRegression...


/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/svm/_base.p

KeyboardInterrupt: 

In [9]:
param = {
            "C": [0.01, 0.1, 1],
            "solver": ['liblinear'],
            "class_weight": [None, 'balanced'],
            "max_iter": [100, 200, 500, 1000]
        }

In [10]:
grid  =GridSearchCV(LogisticRegression(),param_grid=param,cv=5,scoring='accuracy')

In [11]:
grid.fit(X_train_res,y_train_res)
best_results[name] = {
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    }

KeyboardInterrupt: 

In [26]:
data = pd.read_csv("/Users/sarthaktyagi/Desktop/projects /fraudDetection/notebook/newDataForkaggle")

In [27]:
data.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [28]:
data = pd.get_dummies(data,columns=['type'],drop_first=True)

In [29]:
X_train,X_test,y_train,y_test = train_test_split(data.drop(columns=['isFraud']),data.isFraud,stratify=data.isFraud,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(5090096, 10)
(1272524, 10)
(5090096,)
(1272524,)


In [22]:
# Define the mapping
type_mapping = {
    'PAYMENT': 1,
    'TRANSFER': 2,
    'CASH_OUT': 3,
    'DEBIT': 4,
    'CASH_IN': 5
}

# Apply the mapping
data['type'] = data['type'].map(type_mapping)

In [30]:
sm = SMOTE(random_state=42)
X_train_res,y_train_res =sm.fit_resample(X_train,y_train)
from collections import Counter
print("before smote",Counter(y_train))
print("after smote",Counter(y_train_res))

before smote Counter({0: 5083526, 1: 6570})
after smote Counter({0: 5083526, 1: 5083526})


In [31]:
X_train_res.to_csv("trainKaggle",index=False)
y_train_res.to_csv("testKaggle",index=False)

In [16]:
def get_balanced_batch(df, target_col='label', batch_size=200000):
    """
    Returns a balanced batch of data from the given DataFrame.

    Parameters:
    - df: full DataFrame
    - target_col: the name of the label column
    - batch_size: total number of samples in one batch

    Returns:
    - batch_df: a balanced batch DataFrame with total rows = batch_size
    """

    classes = df[target_col].unique()
    n_classes = len(classes)
    samples_per_class = batch_size // n_classes

    balanced_samples = []

    for cls in classes:
        class_df = df[df[target_col] == cls]
        replace = len(class_df) < samples_per_class  # use replacement if not enough samples
        class_sample = class_df.sample(n=samples_per_class, replace=replace, random_state=None)
        balanced_samples.append(class_sample)

    batch_df = pd.concat(balanced_samples).sample(frac=1).reset_index(drop=True)  # shuffle
    return batch_df

In [17]:
data2= X_train_res

In [18]:
data2["isFraud"] = y_train_res

In [19]:
data2.head(3)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest,isFraud
0,15,9914.74,44248.0,34333.26,0.00,0.00,False,False,True,False,0.00,9914.74,0
1,20,6854.53,0.0,0.00,0.00,0.00,False,False,True,False,6854.53,6854.53,0
2,231,361211.80,0.0,0.00,489745.16,850956.95,True,False,False,False,361211.80,0.01,0


In [20]:
subData = get_balanced_batch(df=data2,target_col="isFraud",batch_size=100000)

In [21]:
subData.shape

(100000, 13)

In [22]:
X=subData.drop(columns=['isFraud'])
y = subData.isFraud



In [23]:
X.shape

(100000, 12)

In [19]:
from sklearn.model_selection import GridSearchCV

best_results = {}
param = {
            "C": [0.01, 0.1, 1],
            "solver": ['liblinear'],
            "class_weight": [None, 'balanced'],
            "max_iter": [100, 200, 500, 1000]
        }

grid  =GridSearchCV(LogisticRegression(),param_grid=param,cv=5,scoring='accuracy')
grid.fit(X, y)

best_results[0] = {
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    }

In [15]:
best_results[0]

KeyError: 0

In [24]:
models = {
    "LogisticRegression": {
        "model": LogisticRegression(),
        "param": {
            "C": [0.01, 0.1, 1],
            "solver": ['liblinear'],
            "class_weight": [None, 'balanced'],
            "max_iter": [100, 200, 500, 1000]
        }
    },
    "DecisionTreeClassifier": {
        "model": DecisionTreeClassifier(),
        "param": {
            "criterion": ["gini", "entropy"],
            "max_depth": [5, 10],
            "min_samples_split": [1000, 2000]
        }
    },
    "RandomForestClassifier": {
        "model": RandomForestClassifier(),
        "param": {
            "n_estimators": [50, 100],
            "criterion": ["gini", "entropy"],
            "max_depth": [10, 15],
            "min_samples_split": [1000, 2000]
        }
    }
}

In [25]:
from sklearn.model_selection import GridSearchCV

best_results = {}

# Loop through your models dictionary
for name, m in models.items():
    print(f"Training {name}...")
    grid = GridSearchCV(m['model'], m['param'], cv=5, scoring='accuracy')
    grid.fit(X, y)
    
    # Store best score and params
    best_results[name] = {
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    }

# To view results
from pprint import pprint
pprint(best_results)

Training LogisticRegression...
Training DecisionTreeClassifier...
Training RandomForestClassifier...
{'DecisionTreeClassifier': {'best_params': {'criterion': 'gini',
                                            'max_depth': 10,
                                            'min_samples_split': 1000},
                            'best_score': np.float64(0.9978200000000002)},
 'LogisticRegression': {'best_params': {'C': 0.01,
                                        'class_weight': None,
                                        'max_iter': 100,
                                        'solver': 'liblinear'},
                        'best_score': np.float64(0.8915000000000001)},
 'RandomForestClassifier': {'best_params': {'criterion': 'gini',
                                            'max_depth': 10,
                                            'min_samples_split': 1000,
                                            'n_estimators': 100},
                            'best_score': np.float64(0.997

In [16]:
X.shape

(6362620, 12)

In [26]:
best_model = grid.best_estimator_

In [27]:
print("Best Parameters:", grid.best_params_)


Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 1000, 'n_estimators': 100}


In [28]:
import joblib

# Save
joblib.dump(best_model, "bestRF.joblib")

# Load
model = joblib.load("bestRF.joblib")



In [26]:
y_pred=best_model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))


Accuracy : 0.9999803540051111
Precision: 0.9873493975903614
Recall   : 0.9975654290931223
F1 Score : 0.992431123221314


In [28]:
grid.best_estimator_

RandomForestClassifier(max_depth=10, min_samples_split=1000, n_estimators=50)

In [29]:
import mlflow

In [32]:
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))

metrics ={
    "accuracy":accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score" : f1_score(y_test, y_pred)
}

Accuracy : 0.9999803540051111
Precision: 0.9873493975903614
Recall   : 0.9975654290931223
F1 Score : 0.992431123221314


In [39]:
mlflow.set_experiment("second experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
with mlflow.start_run(run_name= "first"):
    mlflow.log_params(grid.best_params_)
    mlflow.log_metrics(metrics=metrics)
    mlflow.sklearn.log_model(best_model, artifact_path="random_forest")


2025/05/26 21:13:20 INFO mlflow.tracking.fluent: Experiment with name 'second experiment' does not exist. Creating a new experiment.
2025/05/26 21:13:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run first at: http://127.0.0.1:5000/#/experiments/461175113500210767/runs/ecd88949b8384e33bba5e08fcca4df2c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/461175113500210767


In [47]:
best_results['DecisionTreeClassifier']["best_params"]

{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2000}

In [ ]:

best_results

{'LogisticRegression': {'best_score': np.float64(0.907),
  'best_params': {'C': 0.1,
   'class_weight': None,
   'max_iter': 100,
   'solver': 'liblinear'}},
 'DecisionTreeClassifier': {'best_score': np.float64(0.99795),
  'best_params': {'criterion': 'gini',
   'max_depth': 10,
   'min_samples_split': 2000}},
 'RandomForestClassifier': {'best_score': np.float64(0.99774),
  'best_params': {'criterion': 'gini',
   'max_depth': 10,
   'min_samples_split': 1000,
   'n_estimators': 50}}}

In [49]:
dtparams= best_results['DecisionTreeClassifier']['best_params']

In [51]:
dt = DecisionTreeClassifier(**dtparams)
dt.fit(X,y)

DecisionTreeClassifier(max_depth=10, min_samples_split=2000)

In [52]:
dty_pred = dt.predict(X_test)

In [56]:
dtmetrics ={
    "accuracy":accuracy_score(y_test, dty_pred),
    "Precision": precision_score(y_test, dty_pred),
    "Recall": recall_score(y_test, dty_pred),
    "F1 Score" : f1_score(y_test, dty_pred)
}
print(dtmetrics)

{'accuracy': 0.999796467492951, 'Precision': 0.8653643083421331, 'Recall': 0.9975654290931223, 'F1 Score': 0.9267741023466214}


In [58]:
mlflow.set_experiment("second experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run(run_name="decisionTree"):
    mlflow.log_params(dtparams)
    mlflow.log_metrics(dtmetrics)
    mlflow.sklearn.log_model(dt,artifact_path="decision tree classifier")

2025/05/26 21:39:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run decisionTree at: http://127.0.0.1:5000/#/experiments/461175113500210767/runs/f33efa5ed2a842c4858d3edffbf37fe4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/461175113500210767


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,591,6.002549e+04,6.002549e+04,0.0,3.589761e+04,9.592311e+04,True,False,False,False,0.00,-1.000000e-02
1,510,1.671062e+05,1.671062e+05,0.0,7.511967e+06,7.679073e+06,True,False,False,False,0.00,-7.341315e-10
2,354,1.194131e+04,0.000000e+00,0.0,0.000000e+00,0.000000e+00,False,False,True,False,11941.31,1.194131e+04
3,289,1.347307e+06,1.347307e+06,0.0,4.542911e+06,5.890217e+06,True,False,False,False,0.00,5.623837e-03
4,134,1.189206e+05,0.000000e+00,0.0,4.414214e+05,5.603420e+05,True,False,False,False,118920.60,0.000000e+00


In [62]:
data[data['isFraud']==1]

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
2,1,181.00,181.00,0.0,0.00,0.00,1,False,False,False,True,0.0,1.810000e+02
3,1,181.00,181.00,0.0,21182.00,0.00,1,True,False,False,False,0.0,2.136300e+04
251,1,2806.00,2806.00,0.0,0.00,0.00,1,False,False,False,True,0.0,2.806000e+03
252,1,2806.00,2806.00,0.0,26202.00,0.00,1,True,False,False,False,0.0,2.900800e+04
680,1,20128.00,20128.00,0.0,0.00,0.00,1,False,False,False,True,0.0,2.012800e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.0,0.00,339682.13,1,True,False,False,False,0.0,0.000000e+00
6362616,743,6311409.28,6311409.28,0.0,0.00,0.00,1,False,False,False,True,0.0,6.311409e+06
6362617,743,6311409.28,6311409.28,0.0,68488.84,6379898.11,1,True,False,False,False,0.0,1.000000e-02
6362618,743,850002.52,850002.52,0.0,0.00,0.00,1,False,False,False,True,0.0,8.500025e+05


In [66]:
X.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,errorBalanceOrig,errorBalanceDest
0,591,6.002549e+04,6.002549e+04,0.0,3.589761e+04,9.592311e+04,True,False,False,False,0.00,-1.000000e-02
1,510,1.671062e+05,1.671062e+05,0.0,7.511967e+06,7.679073e+06,True,False,False,False,0.00,-7.341315e-10
2,354,1.194131e+04,0.000000e+00,0.0,0.000000e+00,0.000000e+00,False,False,True,False,11941.31,1.194131e+04
3,289,1.347307e+06,1.347307e+06,0.0,4.542911e+06,5.890217e+06,True,False,False,False,0.00,5.623837e-03
4,134,1.189206e+05,0.000000e+00,0.0,4.414214e+05,5.603420e+05,True,False,False,False,118920.60,0.000000e+00


In [68]:
step = 743
amount = 339600
oldbal = 339600	
newbal = 0
oldbal_D =0
newbal_D = 339600
tco = True
td= False
tp= False
tt= False
erBal0=0
erbalD=0


In [71]:
dp = [step,amount,oldbal,newbal,oldbal_D,newbal_D,tco,td,tp,tt,erBal0,erbalD]
ans =best_model.predict([dp])

print(ans)

[1]


/Users/sarthaktyagi/Desktop/projects /fraudDetection/fraudisenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [5]:
newdata = pd.read_csv("/Users/sarthaktyagi/Desktop/projects /fraudDetection/data/PS_20174392719_1491204439457_log.csv")

In [6]:
newdata['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [7]:
best_model

NameError: name 'best_model' is not defined